In [2]:
from PIL import Image
import numpy as np

In [43]:
path  = "./drivers/1/1.png"

size = [25,25]
i  = Image.open(path)
i.thumbnail(size,Image.ANTIALIAS)


a = np.asarray(i)
i = Image.fromarray(a)

#Convert RGBA to RGB by ignoring alpha channel
print i
print a.shape
b = a[:,:,:-1]
print b.shape
i = Image.fromarray(b)
i.show()

<PIL.Image.Image image mode=RGBA size=25x18 at 0x13347E7E8>
(18, 25, 4)
(18, 25, 3)


In [25]:
#Label images
label = path.split("/")[-2]
data  = [b,label]


In [52]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD
import theano
from keras.layers.embeddings import Embedding


In [84]:
X = np.asarray(data[0])
Y = np.asarray([data[1]])


X = X.astype(theano.config.floatX)
Y = Y.astype(theano.config.floatX)


print X.shape
print Y.shape
print X.__class__.__name__
print Y.__class__.__name__


import theano.tensor as T
X = T.as_tensor_variable(X,ndim=3)
Y = T.as_tensor_variable(Y)

print X
print Y

(18, 25, 3)
(1,)
ndarray
ndarray
TensorConstant{[[[ 255.  ...  255.]]]}
TensorConstant{(1,) of 1.0}


In [85]:
input_size = 25*18*3

model = Sequential()
#model.add(Embedding(input_size,input_size))
model.add(Dense(input_size, input_size * 3 ,init='uniform'))
model.add(Dropout(0.5))
model.add(Dense(input_size * 3, input_size * 3, init='uniform'))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(input_size * 3, 2, init='uniform'))
model.add(Activation('softmax'))

print input_size


1350


In [86]:
sgd = SGD(lr=0.1,decay=1e-6,momentum=0.9,nesterov=True)
model.compile(loss='mean_squared_error',optimizer=sgd)
model.fit(X=X,y=Y,batch_size=1,nb_epoch=1,show_accuracy=True,shuffle=True,verbose=2)

TypeError: object of type 'TensorVariable' has no len()

In [ ]:
objective_score = model.evaluate(X,Y,batch_size=1)
classes = model.predict_classes(X,batch_size=1)
proba = model.predict_proba(X,batch_size=1)